In [1]:
from tpot import TPOTRegressor
from tpot import TPOTClassifier

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
biale = pd.read_csv("../input/winequality-white.csv", sep = ";")

In [3]:
biale.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
biale['premium'] = np.where(biale['quality']>=7, 1, 0)
biale["premium"].value_counts()

0    3838
1    1060
Name: premium, dtype: int64

In [5]:
y = biale["quality"]
X = biale.drop(["quality","premium"], axis = 1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
regresor = DecisionTreeRegressor()

In [8]:
regresor.fit(X = X_train, y = y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [9]:
y_pred = regresor.predict(X_test)
mean_squared_error(y_true = y_test, y_pred = y_pred)

0.7155225726654298

In [10]:
tpot_config = {
    'sklearn.tree.DecisionTreeRegressor': {
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21)
    }
}

In [11]:
regresor = TPOTRegressor(verbosity = 1, population_size = 10, generations = 10, config_dict = tpot_config,
                         random_state = 42)
regresor.fit(features = X_train, target = y_train)

Best pipeline: DecisionTreeRegressor(CombineDFs(input_matrix, DecisionTreeRegressor(DecisionTreeRegressor(input_matrix, max_depth=3, min_samples_leaf=14, min_samples_split=18), max_depth=9, min_samples_leaf=20, min_samples_split=2)), max_depth=5, min_samples_leaf=15, min_samples_split=8)


TPOTRegressor(config_dict={'sklearn.tree.DecisionTreeRegressor': {'max_depth': range(1, 11), 'min_samples_split': range(2, 21), 'min_samples_leaf': range(1, 21)}},
       crossover_rate=0.1, cv=5, disable_update_check=False,
       early_stop=None, generations=10, max_eval_time_mins=5,
       max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
       offspring_size=10, periodic_checkpoint_folder=None,
       population_size=10, random_state=42, scoring=None, subsample=1.0,
       verbosity=1, warm_start=False)

In [12]:
regresor.fitted_pipeline_.named_steps

{'featureunion': FeatureUnion(n_jobs=1,
        transformer_list=[('functiontransformer', FunctionTransformer(accept_sparse=False,
           func=<function copy at 0x7f3f639bd0d0>, inv_kw_args=None,
           inverse_func=None, kw_args=None, pass_y='deprecated',
           validate=True)), ('stackingestimator', StackingEstimator(estimator=Pipeline...2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])))],
        transformer_weights=None),
 'decisiontreeregressor': DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=15,
            min_samples_split=8, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')}

In [13]:
regresor.export("../output/fitted-regr-dtr.py")

True

In [14]:
y_pred = regresor.predict(X_test)
mean_squared_error(y_true = y_test, y_pred = y_pred)

0.5600222286218596

In [15]:
regresor = TPOTRegressor(verbosity = 1, population_size = 10, generations = 10, random_state = 42)
regresor.fit(features = X_train, target = y_train)

Best pipeline: RandomForestRegressor(ElasticNetCV(StandardScaler(input_matrix), l1_ratio=0.75, tol=0.01), bootstrap=False, max_features=0.4, min_samples_leaf=2, min_samples_split=14, n_estimators=100)


TPOTRegressor(config_dict={'sklearn.linear_model.ElasticNetCV': {'l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}, 'sklearn.ensemble.ExtraTreesRegressor': {'n_estimato....3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}}}},
       crossover_rate=0.1, cv=5, disable_update_check=False,
       early_stop=None, generations=10, max_eval_time_mins=5,
       max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
       offspring_size=10, periodic_checkpoint_folder=None,
       population_size=10, random_state=42, scoring=None, subsample=1.0,
       verbosity=1, warm_start=False)

In [16]:
regresor.fitted_pipeline_.named_steps

{'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'stackingestimator': StackingEstimator(estimator=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        l1_ratio=0.75, max_iter=1000, n_alphas=100, n_jobs=1,
        normalize=False, positive=False, precompute='auto',
        random_state=None, selection='cyclic', tol=0.01, verbose=0)),
 'randomforestregressor': RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
            max_features=0.4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=14,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False)}

In [17]:
regresor.export("../output/fitted-regr-all.py")

True

In [18]:
y_pred = regresor.predict(X_test)
mean_squared_error(y_true = y_test, y_pred = y_pred)

0.3782467062073013

In [19]:
y = biale["premium"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
klasyfikator = DecisionTreeClassifier()

In [21]:
klasyfikator.fit(X = X_train, y = y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
y_pred = klasyfikator.predict(X_test)
accuracy_score(y_true = y_test, y_pred = y_pred)

0.826221397649969

In [23]:
tpot_config = {
    'sklearn.tree.DecisionTreeClassifier': {
        'criterion': ["gini", "entropy"],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21)
    }
}

In [24]:
klasyfikator = TPOTClassifier(verbosity = 1, population_size = 10, generations = 10, config_dict = tpot_config, 
                              random_state = 42)
klasyfikator.fit(features = X_train, target = y_train)

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=6, min_samples_leaf=17, min_samples_split=5)


TPOTClassifier(config_dict={'sklearn.tree.DecisionTreeClassifier': {'criterion': ['gini', 'entropy'], 'max_depth': range(1, 11), 'min_samples_split': range(2, 21), 'min_samples_leaf': range(1, 21)}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=10, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=10, periodic_checkpoint_folder=None,
        population_size=10, random_state=42, scoring=None, subsample=1.0,
        verbosity=1, warm_start=False)

In [25]:
klasyfikator.fitted_pipeline_.named_steps

{'decisiontreeclassifier': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=17, min_samples_split=5,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best')}

In [26]:
klasyfikator.export("../output/fitted-clas-dtc.py")


True

In [27]:
y_pred = klasyfikator.predict(X_test)
accuracy_score(y_true = y_test, y_pred = y_pred)

0.8002473716759431

In [28]:
klasyfikator = TPOTClassifier(verbosity = 1, population_size = 10, generations = 10, random_state = 42)
klasyfikator.fit(features = X_train, target = y_train)

Best pipeline: GradientBoostingClassifier(FastICA(input_matrix, tol=0.1), learning_rate=0.5, max_depth=6, max_features=0.1, min_samples_leaf=13, min_samples_split=11, n_estimators=100, subsample=0.7000000000000001)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=10, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=10, periodic_checkpoint_folder=None,
        population_size=10, random_state=42, scoring=None, subsample=1.0,
        verbosity=1, warm_start=False)

In [29]:
klasyfikator.fitted_pipeline_.named_steps

{'fastica': FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
     n_components=None, random_state=None, tol=0.1, w_init=None,
     whiten=True),
 'gradientboostingclassifier': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.5, loss='deviance', max_depth=6,
               max_features=0.1, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=13, min_samples_split=11,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               presort='auto', random_state=None,
               subsample=0.7000000000000001, verbose=0, warm_start=False)}

In [30]:
klasyfikator.export("../output/fitted-clas-all.py")

True

In [31]:
y_pred = klasyfikator.predict(X_test)
accuracy_score(y_true = y_test, y_pred = y_pred)

0.8577612863327149

In [32]:
def my_loss(y_true, y_pred): 
    return np.sum(y_true == y_pred) / len(y_true)

my_loss(y_true = y_test, y_pred = y_pred)

0.8577612863327149

In [33]:
my_score = make_scorer(my_loss, greater_is_better=True)

In [34]:
klasyfikator = TPOTClassifier(verbosity = 1, population_size = 10, generations = 10, scoring = my_score)